In [6]:


import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))


def text_to_word_list(text):
    text = text.split()
    return text

def replace_strings(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           u"\u00C0-\u017F"          #latin
                           u"\u2000-\u206F"          #generalPunctuations
                               
                           "]+", flags=re.UNICODE)
    english_pattern=re.compile('[a-zA-Z0-9]+', flags=re.I)
    #latin_pattern=re.compile('[A-Za-z\u00C0-\u00D6\u00D8-\u00f6\u00f8-\u00ff\s]*',)
    
    text=emoji_pattern.sub(r'', text)
    text=english_pattern.sub(r'', text)

    return text

def remove_punctuations(my_str):
    # define punctuation
    punctuations = '''````£|¢|Ñ+-*/=EROero৳০১২৩৪৫৬৭৮৯012–34567•89।!()-[]{};:'"“\’,<>./?@#$%^&*_~‘—॥”‰⚽️✌�￰৷￰'''
    
    no_punct = ""
    for char in my_str:
        if char not in punctuations:
            no_punct = no_punct + char

    # display the unpunctuated string
    return no_punct



def joining(text):
    out=' '.join(text)
    return out

def preprocessing(text):
    out=remove_punctuations(replace_strings(text))
    return out



train_url = '/kaggle/input/80-20ratiofinal/train8020.csv'
test_url = '/kaggle/input/80-20ratiofinal/test8020.csv'
df_train = pd.read_csv(train_url)
df_test = pd.read_csv(test_url)
stop_words_df = pd.read_excel('/kaggle/input/bangla-stopwords/stopwords_bangla.xlsx',index_col=False)
STOPWORDS = set([word.strip() for word in stop_words_df['words']])

df_train['Comment'] = df_train.Comment.apply(lambda x: preprocessing(str(x)))
df_test['Comment'] = df_test.Comment.apply(lambda x:preprocessing(str(x)))
df = pd.concat([df_train,df_test],ignore_index = True)

df_train = df_train.sample(frac=1, random_state=42).reset_index(drop=True)
print(df_train.head(5))

def encode(s):
    d = {
        "Code Switching":0,
        "Grammatical":1,
        "Multiple Errors":2,
        "Spelling":3
    }
    if s in d:
        return d[s]
    else:
        return 4
    
df['Category'] = df.Category.apply(lambda x: encode(x))
df_train['Category'] = df_train.Category.apply(lambda x: encode(x))
df_test['Category'] = df_test.Category.apply(lambda x: encode(x))


      Video ID           Channel name     Time of Publishing  \
0  f-rEHLDfXro  ইতিহাসের অনুসন্ধানে 7G  2022-11-26T10:03:44Z   
1  aLeV04Bz5xk              Channel 24  2023-07-19T13:37:46Z   
2  zBPoDX9TSXA       Cine Fever Bangla  2022-07-15T17:32:43Z   
3  sAoIhoP1zhk                ATN News  2023-06-10T16:32:00Z   
4  jV4gHJYIi34               BigganPiC  2023-02-15T08:00:19Z   

                                               Title          Genre  \
0  ১০০ বছর আগে হারিয়ে যাওয়া গ্রাম কেমন ছিল ইতিহাস...  Miscellaneous   
1  দেশে প্রথমবারের মতো চ্যানেল 24-এর পর্দায় সংবাদ...           News   
2  বড়লোক অনন্ত বর্ষা কে ধুয়ে দিল পরীমনি!দেখুন কি বলল  Entertainment   
3  বিএনপি নির্বাচিত হলে কাকে প্রধানমন্ত্রী করা হব...           News   
4  পৃথিবীর কেন্দ্র Earth core and Earthquake wave...  Miscellaneous   

                                             Comment  Error        Category  \
0  হ্যালো যতগুলা পিক দেখালেন সবগুলা তো মনে হচ্ছে ...      1        Spelling   
1  মানুষের মতো সুন্দর সাবলীল ভ

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input, Attention
from tensorflow.keras.models import Model
# import tensorflow_addons as tfa

# Sample data and labels (replace with your own dataset)
data = df_train['Comment']
labels = df_train['Category']

# Tokenize text data
max_words = 5000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data)
sequences = tokenizer.texts_to_sequences(data)
word_index = tokenizer.word_index
max_sequence_length = 100  # Adjust this based on your data

# Pad sequences
X = pad_sequences(sequences, maxlen=max_sequence_length)

# One-hot encode labels
Y = tf.keras.utils.to_categorical(labels, num_classes=5)

# Create LSTM with Attention model
input_layer = Input(shape=(max_sequence_length,))
embedding_layer = Embedding(max_words, 100)(input_layer)
lstm_layer = LSTM(64, return_sequences=True)(embedding_layer)

# Attention mechanism using TensorFlow Addons
attention = Attention()([lstm_layer, lstm_layer])
attention = tf.keras.layers.GlobalAveragePooling1D()(attention)

output_layer = Dense(5, activation='sigmoid')(attention)
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, Y, epochs=10, batch_size=100, validation_split=0.2)

# Evaluate the model on test data (replace with your test data)
test_data = df_test['Comment'] # Replace with your actual test data
test_labels = df_test['Category']  # Replace with your actual test labels

test_sequences = tokenizer.texts_to_sequences(test_data)
X_test = pad_sequences(test_sequences, maxlen=max_sequence_length)
Y_test = tf.keras.utils.to_categorical(test_labels, num_classes=5)

loss, accuracy = model.evaluate(X_test, Y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

Epoch 1/10
161/161 [==============================] - 27s 149ms/step - loss: 1.3003 - accuracy: 0.5000 - val_loss: 1.4383 - val_accuracy: 0.0000e+00
Epoch 2/10
161/161 [==============================] - 24s 146ms/step - loss: 1.2656 - accuracy: 0.5048 - val_loss: 1.6667 - val_accuracy: 0.0000e+00
Epoch 3/10
161/161 [==============================] - 24s 147ms/step - loss: 1.0613 - accuracy: 0.5717 - val_loss: 1.2293 - val_accuracy: 0.3488
Epoch 4/10
161/161 [==============================] - 24s 148ms/step - loss: 0.8783 - accuracy: 0.6583 - val_loss: 1.2128 - val_accuracy: 0.4586
Epoch 5/10
161/161 [==============================] - 25s 156ms/step - loss: 0.7765 - accuracy: 0.7026 - val_loss: 0.9506 - val_accuracy: 0.6363
Epoch 6/10
161/161 [==============================] - 25s 157ms/step - loss: 0.7000 - accuracy: 0.7369 - val_loss: 1.4678 - val_accuracy: 0.4531
Epoch 7/10
161/161 [==============================] - 24s 150ms/step - loss: 0.6265 - accuracy: 0.7658 - val_loss: 1.4011 

In [3]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
y_pred = model.predict(X_test)

# Calculate macro recall
macro_recall = recall_score(Y_test.argmax(axis=1), y_pred.argmax(axis=1), average='macro')
macro_precision = precision_score(Y_test.argmax(axis=1), y_pred.argmax(axis=1), average='macro')
# Print the results
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}\n  Macro Recall: {:0.3f}\n  Macro Precision: {:0.3f}'.format(loss, accuracy, macro_recall, macro_precision))


157/157 [==============================] - 4s 20ms/step
Test set
  Loss: 1.647
  Accuracy: 0.520
  Macro Recall: 0.367
  Macro Precision: 0.368


In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input, Attention
from tensorflow.keras.models import Model
# import tensorflow_addons as tfa

# Sample data and labels (replace with your own dataset)
data = df_train['Comment']
labels = df_train['Category']

# Tokenize text data
max_words = 10000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data)
sequences = tokenizer.texts_to_sequences(data)
word_index = tokenizer.word_index
max_sequence_length = 100  # Adjust this based on your data

# Pad sequences
X = pad_sequences(sequences, maxlen=max_sequence_length)

# One-hot encode labels
Y = tf.keras.utils.to_categorical(labels, num_classes=5)

# Create LSTM with Attention model
input_layer = Input(shape=(max_sequence_length,))
embedding_layer = Embedding(max_words, 100)(input_layer)
lstm_layer = LSTM(64, return_sequences=True)(embedding_layer)

# Attention mechanism using TensorFlow Addons
attention = Attention()([lstm_layer, lstm_layer])
attention = tf.keras.layers.GlobalAveragePooling1D()(attention)

output_layer = Dense(5, activation='sigmoid')(attention)
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, Y, epochs=10, batch_size=300, validation_split=0.2)

# Evaluate the model on test data (replace with your test data)
test_data = df_test['Comment'] # Replace with your actual test data
test_labels = df_test['Category']  # Replace with your actual test labels

test_sequences = tokenizer.texts_to_sequences(test_data)
X_test = pad_sequences(test_sequences, maxlen=max_sequence_length)
Y_test = tf.keras.utils.to_categorical(test_labels, num_classes=5)

loss, accuracy = model.evaluate(X_test, Y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

Epoch 1/10
54/54 [==============================] - 20s 323ms/step - loss: 1.3350 - accuracy: 0.4945 - val_loss: 1.4507 - val_accuracy: 0.0000e+00
Epoch 2/10
54/54 [==============================] - 17s 312ms/step - loss: 1.2792 - accuracy: 0.5041 - val_loss: 1.5729 - val_accuracy: 0.0000e+00
Epoch 3/10
54/54 [==============================] - 17s 309ms/step - loss: 1.2783 - accuracy: 0.5041 - val_loss: 1.5370 - val_accuracy: 0.0000e+00
Epoch 4/10
54/54 [==============================] - 16s 306ms/step - loss: 1.2763 - accuracy: 0.5041 - val_loss: 1.4725 - val_accuracy: 0.0000e+00
Epoch 5/10
54/54 [==============================] - 16s 303ms/step - loss: 1.2192 - accuracy: 0.5104 - val_loss: 1.2057 - val_accuracy: 0.3346
Epoch 6/10
54/54 [==============================] - 16s 302ms/step - loss: 1.0175 - accuracy: 0.5964 - val_loss: 1.3032 - val_accuracy: 0.3092
Epoch 7/10
54/54 [==============================] - 17s 308ms/step - loss: 0.8075 - accuracy: 0.6968 - val_loss: 1.0351 - val_

In [5]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
y_pred = model.predict(X_test)

# Calculate macro recall
macro_recall = recall_score(Y_test.argmax(axis=1), y_pred.argmax(axis=1), average='macro')
macro_precision = precision_score(Y_test.argmax(axis=1), y_pred.argmax(axis=1), average='macro')
# Print the results
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}\n  Macro Recall: {:0.3f}\n  Macro Precision: {:0.3f}'.format(loss, accuracy, macro_recall, macro_precision))


157/157 [==============================] - 3s 19ms/step
Test set
  Loss: 1.334
  Accuracy: 0.553
  Macro Recall: 0.380
  Macro Precision: 0.391


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input, Attention
from tensorflow.keras.models import Model
# import tensorflow_addons as tfa

# Sample data and labels (replace with your own dataset)
data = df_train['Comment']
labels = df_train['Category']

# Tokenize text data
max_words = 10000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data)
sequences = tokenizer.texts_to_sequences(data)
word_index = tokenizer.word_index
max_sequence_length = 100  # Adjust this based on your data

# Pad sequences
X = pad_sequences(sequences, maxlen=max_sequence_length)

# One-hot encode labels
Y = tf.keras.utils.to_categorical(labels, num_classes=5)

# Create LSTM with Attention model
input_layer = Input(shape=(max_sequence_length,))
embedding_layer = Embedding(max_words, 100)(input_layer)
lstm_layer = LSTM(64, return_sequences=True)(embedding_layer)

# Attention mechanism using TensorFlow Addons
attention = Attention()([lstm_layer, lstm_layer])
attention = tf.keras.layers.GlobalAveragePooling1D()(attention)

output_layer = Dense(5, activation='sigmoid')(attention)
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, Y, epochs=10, batch_size=100, validation_split=0.2)

# Evaluate the model on test data (replace with your test data)
test_data = df_test['Comment'] # Replace with your actual test data
test_labels = df_test['Category']  # Replace with your actual test labels

test_sequences = tokenizer.texts_to_sequences(test_data)
X_test = pad_sequences(test_sequences, maxlen=max_sequence_length)
Y_test = tf.keras.utils.to_categorical(test_labels, num_classes=5)

loss, accuracy = model.evaluate(X_test, Y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

Epoch 1/10
161/161 [==============================] - 27s 151ms/step - loss: 1.2990 - accuracy: 0.5025 - val_loss: 1.5037 - val_accuracy: 0.0000e+00
Epoch 2/10
161/161 [==============================] - 24s 150ms/step - loss: 1.2682 - accuracy: 0.5046 - val_loss: 1.4110 - val_accuracy: 0.0000e+00
Epoch 3/10
161/161 [==============================] - 24s 148ms/step - loss: 1.0380 - accuracy: 0.5822 - val_loss: 1.1848 - val_accuracy: 0.3211
Epoch 4/10
161/161 [==============================] - 24s 150ms/step - loss: 0.7901 - accuracy: 0.6998 - val_loss: 1.4222 - val_accuracy: 0.4025
Epoch 5/10
161/161 [==============================] - 26s 160ms/step - loss: 0.6291 - accuracy: 0.7770 - val_loss: 1.3225 - val_accuracy: 0.5554
Epoch 6/10
161/161 [==============================] - 26s 164ms/step - loss: 0.5159 - accuracy: 0.8232 - val_loss: 1.2142 - val_accuracy: 0.6057
Epoch 7/10
161/161 [==============================] - 27s 168ms/step - loss: 0.4297 - accuracy: 0.8545 - val_loss: 1.5526 

In [7]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
y_pred = model.predict(X_test)

# Calculate macro recall
macro_recall = recall_score(Y_test.argmax(axis=1), y_pred.argmax(axis=1), average='macro')
macro_precision = precision_score(Y_test.argmax(axis=1), y_pred.argmax(axis=1), average='macro')
# Print the results
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}\n  Macro Recall: {:0.3f}\n  Macro Precision: {:0.3f}'.format(loss, accuracy, macro_recall, macro_precision))


157/157 [==============================] - 4s 20ms/step
Test set
  Loss: 1.853
  Accuracy: 0.536
  Macro Recall: 0.387
  Macro Precision: 0.386


In [8]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input, Attention
from tensorflow.keras.models import Model
# import tensorflow_addons as tfa

# Sample data and labels (replace with your own dataset)
data = df_train['Comment']
labels = df_train['Category']

# Tokenize text data
max_words = 10000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data)
sequences = tokenizer.texts_to_sequences(data)
word_index = tokenizer.word_index
max_sequence_length = 100  # Adjust this based on your data

# Pad sequences
X = pad_sequences(sequences, maxlen=max_sequence_length)

# One-hot encode labels
Y = tf.keras.utils.to_categorical(labels, num_classes=5)

# Create LSTM with Attention model
input_layer = Input(shape=(max_sequence_length,))
embedding_layer = Embedding(max_words, 100)(input_layer)
lstm_layer = LSTM(64, return_sequences=True)(embedding_layer)

# Attention mechanism using TensorFlow Addons
attention = Attention()([lstm_layer, lstm_layer])
attention = tf.keras.layers.GlobalAveragePooling1D()(attention)

output_layer = Dense(5, activation='sigmoid')(attention)
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, Y, epochs=10, batch_size=500, validation_split=0.2)

# Evaluate the model on test data (replace with your test data)
test_data = df_test['Comment'] # Replace with your actual test data
test_labels = df_test['Category']  # Replace with your actual test labels

test_sequences = tokenizer.texts_to_sequences(test_data)
X_test = pad_sequences(test_sequences, maxlen=max_sequence_length)
Y_test = tf.keras.utils.to_categorical(test_labels, num_classes=5)

loss, accuracy = model.evaluate(X_test, Y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

Epoch 1/10
33/33 [==============================] - 19s 489ms/step - loss: 1.3583 - accuracy: 0.4959 - val_loss: 1.4784 - val_accuracy: 0.0000e+00
Epoch 2/10
33/33 [==============================] - 15s 456ms/step - loss: 1.2795 - accuracy: 0.5041 - val_loss: 1.4536 - val_accuracy: 0.0000e+00
Epoch 3/10
33/33 [==============================] - 15s 462ms/step - loss: 1.2788 - accuracy: 0.5041 - val_loss: 1.4352 - val_accuracy: 0.0000e+00
Epoch 4/10
33/33 [==============================] - 15s 454ms/step - loss: 1.2780 - accuracy: 0.5041 - val_loss: 1.5235 - val_accuracy: 0.0000e+00
Epoch 5/10
33/33 [==============================] - 16s 484ms/step - loss: 1.2759 - accuracy: 0.5041 - val_loss: 1.4773 - val_accuracy: 0.0000e+00
Epoch 6/10
33/33 [==============================] - 16s 499ms/step - loss: 1.2646 - accuracy: 0.5041 - val_loss: 1.4943 - val_accuracy: 0.0000e+00
Epoch 7/10
33/33 [==============================] - 16s 473ms/step - loss: 1.1893 - accuracy: 0.5114 - val_loss: 1.532

In [9]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
y_pred = model.predict(X_test)

# Calculate macro recall
macro_recall = recall_score(Y_test.argmax(axis=1), y_pred.argmax(axis=1), average='macro')
macro_precision = precision_score(Y_test.argmax(axis=1), y_pred.argmax(axis=1), average='macro')
# Print the results
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}\n  Macro Recall: {:0.3f}\n  Macro Precision: {:0.3f}'.format(loss, accuracy, macro_recall, macro_precision))


157/157 [==============================] - 4s 20ms/step
Test set
  Loss: 1.232
  Accuracy: 0.526
  Macro Recall: 0.364
  Macro Precision: 0.389


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input, Attention
from tensorflow.keras.models import Model
# import tensorflow_addons as tfa

# Sample data and labels (replace with your own dataset)
data = df_train['Comment']
labels = df_train['Category']

# Tokenize text data
max_words = 10000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data)
sequences = tokenizer.texts_to_sequences(data)
word_index = tokenizer.word_index
max_sequence_length = 100  # Adjust this based on your data

# Pad sequences
X = pad_sequences(sequences, maxlen=max_sequence_length)

# One-hot encode labels
Y = tf.keras.utils.to_categorical(labels, num_classes=5)

# Create LSTM with Attention model
input_layer = Input(shape=(max_sequence_length,))
embedding_layer = Embedding(max_words, 100)(input_layer)
lstm_layer = LSTM(64, return_sequences=True)(embedding_layer)

# Attention mechanism using TensorFlow Addons
attention = Attention()([lstm_layer, lstm_layer])
attention = tf.keras.layers.GlobalAveragePooling1D()(attention)

output_layer = Dense(5, activation='sigmoid')(attention)
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, Y, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model on test data (replace with your test data)
test_data = df_test['Comment'] # Replace with your actual test data
test_labels = df_test['Category']  # Replace with your actual test labels

test_sequences = tokenizer.texts_to_sequences(test_data)
X_test = pad_sequences(test_sequences, maxlen=max_sequence_length)
Y_test = tf.keras.utils.to_categorical(test_labels, num_classes=5)

loss, accuracy = model.evaluate(X_test, Y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

Epoch 1/10
503/503 [==============================] - 41s 75ms/step - loss: 1.2881 - accuracy: 0.5025 - val_loss: 1.5428 - val_accuracy: 0.0000e+00
Epoch 2/10
503/503 [==============================] - 40s 80ms/step - loss: 1.0651 - accuracy: 0.5712 - val_loss: 1.4009 - val_accuracy: 0.0876
Epoch 3/10
503/503 [==============================] - 38s 75ms/step - loss: 0.7780 - accuracy: 0.7017 - val_loss: 1.1449 - val_accuracy: 0.5671
Epoch 4/10
503/503 [==============================] - 37s 74ms/step - loss: 0.5846 - accuracy: 0.7873 - val_loss: 1.3489 - val_accuracy: 0.5345
Epoch 5/10
503/503 [==============================] - 38s 76ms/step - loss: 0.4435 - accuracy: 0.8371 - val_loss: 1.4863 - val_accuracy: 0.5738
Epoch 6/10
503/503 [==============================] - 38s 75ms/step - loss: 0.3450 - accuracy: 0.8772 - val_loss: 2.0660 - val_accuracy: 0.4680
Epoch 7/10
503/503 [==============================] - 38s 75ms/step - loss: 0.2746 - accuracy: 0.9027 - val_loss: 1.9246 - val_accur

In [5]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
y_pred = model.predict(X_test)

# Calculate macro recall
macro_recall = recall_score(Y_test.argmax(axis=1), y_pred.argmax(axis=1), average='macro')
macro_precision = precision_score(Y_test.argmax(axis=1), y_pred.argmax(axis=1), average='macro')
# Print the results
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}\n  Macro Recall: {:0.3f}\n  Macro Precision: {:0.3f}'.format(loss, accuracy, macro_recall, macro_precision))


157/157 [==============================] - 3s 21ms/step
Test set
  Loss: 2.343
  Accuracy: 0.531
  Macro Recall: 0.377
  Macro Precision: 0.381


In [7]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input, Attention
from tensorflow.keras.models import Model
# import tensorflow_addons as tfa

# Sample data and labels (replace with your own dataset)
data = df_train['Comment']
labels = df_train['Category']

# Tokenize text data
max_words = 10000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data)
sequences = tokenizer.texts_to_sequences(data)
word_index = tokenizer.word_index
max_sequence_length = 100  # Adjust this based on your data

# Pad sequences
X = pad_sequences(sequences, maxlen=max_sequence_length)

# One-hot encode labels
Y = tf.keras.utils.to_categorical(labels, num_classes=5)

# Create LSTM with Attention model
input_layer = Input(shape=(max_sequence_length,))
embedding_layer = Embedding(max_words, 100)(input_layer)
lstm_layer = LSTM(64, return_sequences=True)(embedding_layer)

# Attention mechanism using TensorFlow Addons
attention = Attention()([lstm_layer, lstm_layer])
attention = tf.keras.layers.GlobalAveragePooling1D()(attention)

output_layer = Dense(5, activation='sigmoid')(attention)
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, Y, epochs=10, batch_size=300, validation_split=0.2)

# Evaluate the model on test data (replace with your test data)
test_data = df_test['Comment'] # Replace with your actual test data
test_labels = df_test['Category']  # Replace with your actual test labels

test_sequences = tokenizer.texts_to_sequences(test_data)
X_test = pad_sequences(test_sequences, maxlen=max_sequence_length)
Y_test = tf.keras.utils.to_categorical(test_labels, num_classes=5)

loss, accuracy = model.evaluate(X_test, Y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

Epoch 1/10
54/54 [==============================] - 21s 344ms/step - loss: 1.3160 - accuracy: 0.3915 - val_loss: 1.2574 - val_accuracy: 0.3869
Epoch 2/10
54/54 [==============================] - 17s 322ms/step - loss: 1.2610 - accuracy: 0.3976 - val_loss: 1.2539 - val_accuracy: 0.3998
Epoch 3/10
54/54 [==============================] - 18s 327ms/step - loss: 1.2585 - accuracy: 0.4058 - val_loss: 1.2518 - val_accuracy: 0.3998
Epoch 4/10
54/54 [==============================] - 17s 319ms/step - loss: 1.2446 - accuracy: 0.4318 - val_loss: 1.2055 - val_accuracy: 0.4730
Epoch 5/10
54/54 [==============================] - 18s 325ms/step - loss: 1.0881 - accuracy: 0.5409 - val_loss: 1.0689 - val_accuracy: 0.5479
Epoch 6/10
54/54 [==============================] - 17s 322ms/step - loss: 0.8550 - accuracy: 0.6734 - val_loss: 1.0443 - val_accuracy: 0.5763
Epoch 7/10
54/54 [==============================] - 17s 323ms/step - loss: 0.6898 - accuracy: 0.7539 - val_loss: 1.0661 - val_accuracy: 0.5910

In [8]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
y_pred = model.predict(X_test)

# Calculate macro recall
macro_recall = recall_score(Y_test.argmax(axis=1), y_pred.argmax(axis=1), average='macro')
macro_precision = precision_score(Y_test.argmax(axis=1), y_pred.argmax(axis=1), average='macro')
# Print the results
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}\n  Macro Recall: {:0.3f}\n  Macro Precision: {:0.3f}'.format(loss, accuracy, macro_recall, macro_precision))


157/157 [==============================] - 4s 20ms/step
Test set
  Loss: 1.310
  Accuracy: 0.566
  Macro Recall: 0.379
  Macro Precision: 0.393


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [9]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input, Attention
from tensorflow.keras.models import Model
# import tensorflow_addons as tfa

# Sample data and labels (replace with your own dataset)
data = df_train['Comment']
labels = df_train['Category']

# Tokenize text data
max_words = 10000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data)
sequences = tokenizer.texts_to_sequences(data)
word_index = tokenizer.word_index
max_sequence_length = 100  # Adjust this based on your data

# Pad sequences
X = pad_sequences(sequences, maxlen=max_sequence_length)

# One-hot encode labels
Y = tf.keras.utils.to_categorical(labels, num_classes=5)

# Create LSTM with Attention model
input_layer = Input(shape=(max_sequence_length,))
embedding_layer = Embedding(max_words, 100)(input_layer)
lstm_layer = LSTM(64, return_sequences=True)(embedding_layer)

# Attention mechanism using TensorFlow Addons
attention = Attention()([lstm_layer, lstm_layer])
attention = tf.keras.layers.GlobalAveragePooling1D()(attention)

output_layer = Dense(5, activation='sigmoid')(attention)
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, Y, epochs=10, batch_size=100, validation_split=0.2)

# Evaluate the model on test data (replace with your test data)
test_data = df_test['Comment'] # Replace with your actual test data
test_labels = df_test['Category']  # Replace with your actual test labels

test_sequences = tokenizer.texts_to_sequences(test_data)
X_test = pad_sequences(test_sequences, maxlen=max_sequence_length)
Y_test = tf.keras.utils.to_categorical(test_labels, num_classes=5)

loss, accuracy = model.evaluate(X_test, Y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

Epoch 1/10
161/161 [==============================] - 28s 160ms/step - loss: 1.2844 - accuracy: 0.3931 - val_loss: 1.2578 - val_accuracy: 0.3998
Epoch 2/10
161/161 [==============================] - 24s 151ms/step - loss: 1.2578 - accuracy: 0.4063 - val_loss: 1.2336 - val_accuracy: 0.3998
Epoch 3/10
161/161 [==============================] - 25s 153ms/step - loss: 1.0764 - accuracy: 0.5454 - val_loss: 1.0233 - val_accuracy: 0.5663
Epoch 4/10
161/161 [==============================] - 24s 152ms/step - loss: 0.8017 - accuracy: 0.6990 - val_loss: 1.0228 - val_accuracy: 0.5960
Epoch 5/10
161/161 [==============================] - 25s 157ms/step - loss: 0.6353 - accuracy: 0.7746 - val_loss: 1.0625 - val_accuracy: 0.6042
Epoch 6/10
161/161 [==============================] - 25s 153ms/step - loss: 0.5116 - accuracy: 0.8266 - val_loss: 1.1897 - val_accuracy: 0.6002
Epoch 7/10
161/161 [==============================] - 25s 154ms/step - loss: 0.4175 - accuracy: 0.8595 - val_loss: 1.2760 - val_ac

In [10]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
y_pred = model.predict(X_test)

# Calculate macro recall
macro_recall = recall_score(Y_test.argmax(axis=1), y_pred.argmax(axis=1), average='macro')
macro_precision = precision_score(Y_test.argmax(axis=1), y_pred.argmax(axis=1), average='macro')
# Print the results
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}\n  Macro Recall: {:0.3f}\n  Macro Precision: {:0.3f}'.format(loss, accuracy, macro_recall, macro_precision))

157/157 [==============================] - 4s 21ms/step
Test set
  Loss: 1.741
  Accuracy: 0.555
  Macro Recall: 0.386
  Macro Precision: 0.391
